# Your Guide to Agent-Factory!

Welcome to your entry point for using Agent-Factory!

This notebook will guide you through the process of creating, running and, perhaps most importantly, evaluating your first agent. As a "[hello-world](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program)" example, we will build an agent that summarizes text content from a given webpage URL. This notebook includes all execution outputs so you can follow along without having to execute it. If you prefer to start fresh and try it yourself, you can select from the Jupyter Notebook's menu: `Edit -> Clear All Outputs`.

But, before we begin:

### What is Agent-Factory?

Agent-Factory is a framework that enables users to build and deploy AI agents through natural language prompts, lowering the technical barrier of entry for non-AI experts.

### How does it work?

The main concepts you will need to understand to get you started are:

- **Task**: The high-level description of what you want to achieve with your agent. In this case, we want to summarize text content from a given webpage URL.

- **Manufacturing Agent**: The agent that will help you create your own agents. Think of it as you own personal alchemist-druid! You submit your vision of your ideal minion-assistant that you need help with for a very specific task, and the Manufacturing Agent will fulfill your wish.

  - _In practice:_ A server, running on the background, that receives requests as a natural language prompt, and builds the Target Agent.

- **Target Agent**: The agent that the Manufacturing Agent creates for you, tailored to do the best that it can to complete your task.
It is a fully functional agent that can be run independently, anywhere you like, and it will have its own set of very specific capabilities and requirements, tailored to the specific task you requested.

  - _In practice:_ A directory containing of Python code (agent.py) that implements the Target Agent, along with all the files necessary for the agent to be successfully run (`tools`, `requirements.txt`, `agent_parameters.json`, etc.).

- **Criteria Agent**: The agent that can help you generate a set of criteria, aka **Evaluation Case**, to evaluate the Target Agent. You can imagine these criteria as a check-list for the Target Agent to ensure that has completed the task you requested as intended. **NOTE:** these criteria can also be manually defined by you. A recommended flow would be that you first generate a set with the Criteria Agent, and then you review and refine them to your needs.

  - _In practice:_ A structured JSON file that contains the evaluation criteria, such as the input data, expected output, and any other relevant information needed to evaluate the Target Agent.

- **Agent Judge**: The agent that will execute the Target Agent against the Evaluation Case and provide a score based on how well it performed. This is the final step to ensure that the Target Agent is capable of completing the task you requested.

  - _In practice:_ The Agent Judge will read through the Target Agent's output (traces) and compare it to all the criteria defined in the Evaluation Case, scoring whether the criteria were met (found in the output) or not.

### Installation

Before running this notebook, ensure that you have installed Agent-Factory, by following the instructions in the [Installation Guide](getting-started/installation.md). Don't forget to also create a .env file with your OpenAI and Tavily API keys, as they are necessary for this notebook.

### Setup

#### Jupyter Notebook

To start this notebook, in a terminal with the project's virtual environment activated, run:
```bash
uv run --with jupyter jupyter lab
```
and you will now be able to run the commands below from within this notebook.

#### Agent-Factory Server

In another terminal (not inside this notebook), move to the src/agent-factory directory and start the agent-factory server in `--nochat` mode:

```bash
cd src/agent_factory && uv run . --host 0.0.0.0 --port 8080 --nochat
```

This will prepare the Manufacturing Agent to receive requests and build any Target Agent you ask.
The `--nochat` flag is used so that the Manufacturing Agent "one-shots" your request, meaning it will not engage in a back-and-forth conversation with you, but rather will try to fulfill your request in one go.


### Let's build!



In [1]:
# Necessary to run the Target Agent within a notebook environment
import nest_asyncio

nest_asyncio.apply()

In [2]:
# This notebook is in under the /docs directory, so we need to go up one level to run the agent-factory command
%cd ../

/home/kostis/MZAI/agent-factory


Let's ask the Manufacturing Agent to create the Target Agent for us by providing our prompt and an output directory.


In [3]:
# The --active flag for uv run enables to use the preconfigured active virtual env instead of jupyter's environment
!uv run --active agent-factory "Summarize text content from a given webpage URL" --output_dir hello_world

/home/kostis/.cache/uv/archive-v0/tJFHV0c7OHrqMpkwp_PhM/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
[08/12/25 13:56:33] INFO     A2AClient initialized.        ]8;id=160684;file:///home/kostis/MZAI/agent-factory/src/agent_factory/agent_generator.py\agent_generator.py]8;;\:]8;id=597528;file:///home/kostis/MZAI/agent-factory/src/agent_factory/agent_generator.py#50\50]8;;\
                    INFO     No request ID provided,          ]8;id=78310;file:///home/kostis/MZAI/agent-factory/src/agent_factory/utils/client_utils.py\client_utils.py]8;;\:]8;id=659058;file:///home/kostis/MZAI/agent-factory/src/agent_factory/utils/client_utils.py#42\42]8;;\
                             generating a new one                              

Now that the Target Agent has been created, let's navigate to the directory where it was created and see the files it generated.

In [4]:
%cd generated_workflows/hello_world
%ls

/home/kostis/MZAI/agent-factory/generated_workflows/hello_world
agent_parameters.json  agent.py  README.md  requirements.txt  tools/


Next step: let's run it!

We are using `uv` to explicitly define the Python version we want to use, in this case, Python 3.13, which packages to install from the generated requirements.txt file, and,
finally, the input for the Target Agent, i.e. the URL we want to summarize.

In [5]:
!uv run --with-requirements requirements.txt --python 3.13 python agent.py --url https://blog.mozilla.ai/introducing-any-llm-a-unified-api-to-access-any-llm-provider/

/home/kostis/MZAI/agent-factory/.venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
╭──────────────────────────────── CALL_LLM: o3 ────────────────────────────────╮
│ ╭─ INPUT ──────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                        │ │
│ │   {                                                                      │ │
│ │     "role": "system",                                                    │ │
│ │     "content": "\nYou are an assistant that follows this concise multi–s │ │
│ │   },                                                                     │ │
│ │   {                                                                      │ │
│ │     "conte

Great, this summary is looking pretty good! 🎉

But, how do we know whether the Target Agent actually did what we had in mind? For example, did it actually visit the URL we provided, or did it just hallucinate and completely made up the summary? 🤔

### Evaluation time!

In order to ensure that our Target Agent acts according to our requirements, we will build an Evaluation Case with certain criteria the Target Agent must meet in order to be considered successful.
As we mentioned in the beginning, this can be done either manually or automatically from the Criteria Agent.
To simplify the process, we recommend to first use the Criteria Agent to auto-generate a few relevant criteria for us,
and if we are not satisfied with the results, we can refine the criteria later.

In [6]:
# Let's navigate back to the root directory of the project, so we can run the Criteria Agent.
%cd ../../

/home/kostis/MZAI/agent-factory


In [7]:
!uv run --active -m eval.generate_evaluation_case generated_workflows/hello_world

/home/kostis/.cache/uv/archive-v0/tJFHV0c7OHrqMpkwp_PhM/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
Secure MCP Filesystem Server running on stdio
Allowed directories: [ '/app' ]
╭───────────────────────────── CALL_LLM: gpt-4.1 ──────────────────────────────╮
│ ╭─ INPUT ──────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                        │ │
│ │   {                                                                      │ │
│ │     "role": "system",                                                    │ │
│ │     "content": "\nThe generated_workflow_dir is generated_workflows/hell │ │
│ │   },                                                                     │ │

We can see that the Criteria Agent has created a file called `evaluation_case.json` in the `generated_workflows/hello_world` directory. Let's open it to see the criteria it generated for us.

In [16]:
import json
from pathlib import Path

with Path.open("generated_workflows/hello_world/evaluation_case.json", "r") as f:
    eval_case = json.load(f)
    print(json.dumps(eval_case["criteria"], indent=4))

[
    "Ensure that the agent uses the visit_webpage tool to fetch the webpage content for the given URL.",
    "Ensure that the agent parses only the primary textual content from the fetched Markdown, explicitly ignoring navigation menus, advertisements, footers, scripts, and other non-informational elements.",
    "Ensure that the summary field in the output is a clear, English summary of approximately 100 to 150 words that accurately conveys the main ideas, arguments, and conclusions from the webpage, without adding external information or speculation.",
    "Ensure that, if the webpage cannot be fetched or has insufficient textual content, the agent explains the issue succinctly within the summary field, as per instructions.",
    "Ensure that the agent returns output strictly as a JSON object matching the StructuredOutput schema, containing exactly two fields: url (the original input URL) and summary (the generated summary).",
    "Verify that the url field in the output exactly ma

**Remember** that you can edit this file according to you needs! If a certain criteria is not relevant, you can remove it. Or if something is missing, you can just add it to the file.

For our use case, these criteria are looking pretty good, so let's use them as is.

The next step is to run the Agent Judge, which will execute the Target Agent against the Evaluation Case and provide a score based on how well it performed.

In [8]:
!uv run --active -m eval.run_generated_agent_evaluation generated_workflows/hello_world

/home/kostis/.cache/uv/archive-v0/tJFHV0c7OHrqMpkwp_PhM/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
[08/12/25 14:06:32] INFO     Successfully   ]8;id=100339;file:///home/kostis/MZAI/agent-factory/eval/run_generated_agent_evaluation.py\run_generated_agent_evaluation.py]8;;\:]8;id=44959;file:///home/kostis/MZAI/agent-factory/eval/run_generated_agent_evaluation.py#48\48]8;;\
                             loaded                                             
                             evaluation                                         
                             case from:                                         
                             generated_work                                     
                             flows/he

### Results & Next steps

As we can see from the Agent Judge's trace our Target Agent got 6 out 8 of the evaluation criteria! To further inspect where it failed, we can view the file: `generated_workflows/hello_world/evaluation_results.json`. 

In [17]:
with Path.open("generated_workflows/hello_world/evaluation_results.json", "r") as f:
    evaluation_results = json.load(f)
    print(json.dumps(evaluation_results, indent=4))

{
    "obtained_score": 6,
    "max_score": 8,
    "results": [
        {
            "passed": true,
            "reasoning": "The agent trace shows that the visit_webpage tool was used with the correct URL before generating a summary. This confirms the agent properly fetched the webpage content using the specified tool as required."
        },
        {
            "passed": true,
            "reasoning": "The agent output only summarized the main textual content of the blog post and ignored menus, advertisements, footers, and unrelated sections. The result matches the expected requirement and maintains focus on the primary informational content."
        },
        {
            "passed": false,
            "reasoning": "The output does not include a 'summary' field containing the required clear, 100-150 word English summary of the specified webpage. Therefore, it does not satisfy the evaluation criteria as specified in the question or schema."
        },
        {
            "pass

For example, we can see that one of the criteria that the Criteria Agent generated, but our Target Agent failed at was:

> The total token usage is 3777, which is above the 1000 token limit for intermediate reasoning and tool usage.

Now depending on your needs, this might be a reasonable ask, or too restricting. If it's a reasonable requirement, you could copy-paste this criteria in the initial prompt in the Manufacturing Agent so that the Target Agent is implemented with this requirement in mind. Otherwise, if it's too restricting, you could remove it from the Evaluation Case.

***Final Note***: Building AI Agents is about understanding your needs, clearly defining them, and iterating on your Agents when those needs change. The evaluation component of agent-factory is a crucial step to ensure the agents you build are always aligned to your needs.